# TP2 - Part 1: From Text to Vectors

**Day 2 - AI for Sciences Winter School**

**Instructor:** Raphael Cousin

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/ai_for_sciences/blob/main/day2/tp2_part1.ipynb)

---

## Objectives

By the end of this practical, you will understand:

1. **The pipeline**: Raw data → Tokens → Numbers → Embeddings
2. **What embeddings are**: Dense vector representations that capture meaning
3. **Why embeddings matter**: Similar things have similar vectors
4. **How to measure similarity**: Cosine similarity and distance in embedding space

---

# The Big Picture: Why Do We Need Embeddings?

**The fundamental problem**: Neural networks only understand numbers, not text, molecules, or proteins.

**The solution**: Convert everything into **vectors** (lists of numbers).

But not just any vectors—we want vectors where **similar things are close together**.

```
"King" → [0.2, -0.4, 0.8, 0.1, ...] (384 numbers)
"Queen" → [0.3, -0.3, 0.7, 0.2, ...] (384 numbers)  ← Close to "King"!
"Banana" → [-0.5, 0.9, -0.2, 0.6, ...] (384 numbers)  ← Far from both
```

This is the key insight of **embeddings**: meaning becomes geometry.

---

# Part 1: The Complete Pipeline

Let's trace the journey from raw text to embeddings:

```
┌─────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────────┐
│  Raw Text   │ →  │   Tokens     │ →  │  Token IDs   │ →  │    Embeddings    │
│  (string)   │    │  (subwords)  │    │  (integers)  │    │ (dense vectors)  │
└─────────────┘    └──────────────┘    └──────────────┘    └──────────────────┘

"The cat sat"  →  ["The", "cat", "sat"] →  [464, 3797, 3332]  →  [[0.1, -0.2, ...], 
                                                                   [0.3, 0.5, ...],
                                                                   [-0.1, 0.4, ...]]
```

**Key question**: What makes a good embedding?

## Setup

In [ ]:
# Install required packages
!pip install -q transformers sentence-transformers torch numpy matplotlib scikit-learn

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

print("Setup complete!")

---

# Part 2: What Are Embeddings?

An **embedding** is a vector representation where:
- Each dimension captures some aspect of meaning
- Similar items have similar vectors
- Operations on vectors correspond to operations on meaning

## The Naive Approach: One-Hot Encoding

The simplest way to represent tokens as numbers:

In [ ]:
# Simple vocabulary
vocab = ["cat", "dog", "fish", "bird", "tree"]

# One-hot encoding: each word is a vector with one "1" and rest "0"s
one_hot = {
    "cat":  [1, 0, 0, 0, 0],
    "dog":  [0, 1, 0, 0, 0],
    "fish": [0, 0, 1, 0, 0],
    "bird": [0, 0, 0, 1, 0],
    "tree": [0, 0, 0, 0, 1],
}

print("One-hot representations:")
for word, vec in one_hot.items():
    print(f"  {word:6} → {vec}")

### 🤔 Question 1

Look at the one-hot vectors above.

1. What is the distance between "cat" and "dog"?
2. What is the distance between "cat" and "tree"?
3. Is this what we want? Why or why not?

*Think about it before running the next cell.*

In [ ]:
# Let's compute distances
cat = np.array(one_hot["cat"])
dog = np.array(one_hot["dog"])
tree = np.array(one_hot["tree"])

# Euclidean distance
dist_cat_dog = np.linalg.norm(cat - dog)
dist_cat_tree = np.linalg.norm(cat - tree)

print(f"Distance between 'cat' and 'dog':  {dist_cat_dog:.2f}")
print(f"Distance between 'cat' and 'tree': {dist_cat_tree:.2f}")
print()
print("→ With one-hot encoding, ALL words are equally distant from each other!")
print("→ We lose the semantic relationship: cat and dog are both animals.")

## The Better Approach: Dense Embeddings

Instead of sparse one-hot vectors, we use **dense vectors** where:
- Each dimension can have any real value
- Dimensions capture abstract features (not explicitly named)
- **Learning** discovers which dimensions matter

Imagine these learned embeddings:

```
             [is_animal, has_fur, can_fly, is_pet, ...]
"cat"   →   [   0.9,      0.8,     0.0,    0.7,   ...]
"dog"   →   [   0.9,      0.9,     0.0,    0.9,   ...]
"bird"  →   [   0.8,      0.0,     0.9,    0.5,   ...]
"tree"  →   [   0.0,      0.0,     0.0,    0.0,   ...]
```

Now "cat" and "dog" are close because they share features!

In [ ]:
# Simulated "learned" embeddings (just for illustration)
# In reality, these are learned from data by neural networks

embeddings = {
    "cat":   np.array([0.9, 0.8, 0.0, 0.7]),
    "dog":   np.array([0.9, 0.9, 0.0, 0.9]),
    "fish":  np.array([0.7, 0.0, 0.0, 0.4]),
    "bird":  np.array([0.8, 0.0, 0.9, 0.5]),
    "tree":  np.array([0.0, 0.0, 0.0, 0.0]),
}

cat = embeddings["cat"]
dog = embeddings["dog"]
tree = embeddings["tree"]

# Now distances reflect semantic similarity!
print("With dense embeddings:")
print(f"  Distance 'cat' ↔ 'dog':  {np.linalg.norm(cat - dog):.2f}  (close - both pets!)")
print(f"  Distance 'cat' ↔ 'tree': {np.linalg.norm(cat - tree):.2f}  (far - different things)")

---

# Part 3: Cosine Similarity — The Standard Measure

In practice, we don't use Euclidean distance for embeddings. We use **cosine similarity**.

$$\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$$

This measures the **angle** between vectors, not the distance.

**Why cosine similarity?**
- Scale-invariant: [1, 2, 3] and [2, 4, 6] have cosine similarity = 1.0
- Range: -1 (opposite) to 1 (identical direction)
- Works better when embeddings have different magnitudes

In [ ]:
def cosine_sim(a, b):
    """Compute cosine similarity between two vectors."""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Compare all pairs
words = list(embeddings.keys())
print("Cosine similarities between words:\n")

for i, w1 in enumerate(words):
    for w2 in words[i+1:]:
        sim = cosine_sim(embeddings[w1], embeddings[w2])
        bar = "█" * int(sim * 20) if sim > 0 else ""
        print(f"  {w1:5} ↔ {w2:5}: {sim:+.2f}  {bar}")

### 🤔 Question 2

Looking at the cosine similarities above:

1. Which pair of words has the highest similarity? Does this make sense?
2. Why is "tree" so different from all animals?
3. Why might "fish" be somewhat similar to "cat" and "dog" (both are pets)?

---

# Part 4: Real Embeddings from Pre-trained Models

The embeddings above were hand-crafted. In practice, we use **pre-trained models** that learned embeddings from massive datasets.

Let's use a real embedding model: `sentence-transformers/all-MiniLM-L6-v2`

This model:
- Was trained on over 1 billion sentence pairs
- Produces 384-dimensional embeddings
- Captures semantic meaning surprisingly well

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

print(f"Model loaded!")
print(f"Embedding dimension: {model.get_sentence_embedding_dimension()}")

In [ ]:
# Let's embed some sentences
sentences = [
    "The cat is sleeping on the couch.",
    "A dog is resting on the sofa.",
    "Machine learning transforms data into insights.",
    "Deep learning models can recognize images.",
    "The weather is sunny today.",
]

# Compute embeddings
embeddings = model.encode(sentences)

print(f"Shape of embeddings: {embeddings.shape}")
print(f"→ {len(sentences)} sentences, each represented by {embeddings.shape[1]} numbers")
print()
print("First embedding (first 10 values):")
print(f"  {embeddings[0][:10]}...")

### Let's Visualize the Similarity Matrix

A similarity matrix shows how similar each sentence is to every other sentence.

In [ ]:
# Compute pairwise cosine similarities
similarities = cosine_similarity(embeddings)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))

im = ax.imshow(similarities, cmap='Blues', vmin=0, vmax=1)
plt.colorbar(im, label='Cosine Similarity')

# Labels
short_labels = [s[:25] + "..." if len(s) > 25 else s for s in sentences]
ax.set_xticks(range(len(sentences)))
ax.set_yticks(range(len(sentences)))
ax.set_xticklabels(short_labels, rotation=45, ha='right', fontsize=9)
ax.set_yticklabels(short_labels, fontsize=9)

# Add values
for i in range(len(sentences)):
    for j in range(len(sentences)):
        ax.text(j, i, f'{similarities[i,j]:.2f}', ha='center', va='center', fontsize=8)

ax.set_title('Sentence Similarity Matrix', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

### 🤔 Question 3

Look at the similarity matrix above:

1. Which two sentences are most similar (besides the diagonal)?
2. Why are sentences 1 and 2 similar? (Hint: read them again)
3. Why are sentences 3 and 4 similar?
4. Why is sentence 5 different from all others?

---

# Part 5: The Embedding Space — Visualizing High Dimensions

Our embeddings have 384 dimensions. We can't visualize 384D directly, but we can project them to 2D using **PCA** (Principal Component Analysis).

PCA finds the 2 directions that capture the most variance in the data.

In [ ]:
from sklearn.decomposition import PCA

# More diverse sentences for visualization
sentences_diverse = [
    # Animals
    "The cat is sleeping.",
    "Dogs are loyal companions.",
    "Fish swim in the ocean.",
    # Technology
    "Computers process data quickly.",
    "Software engineers write code.",
    "Artificial intelligence is advancing.",
    # Nature
    "Mountains are covered in snow.",
    "The forest is full of trees.",
    "Rivers flow to the sea.",
]

categories = ["Animals"] * 3 + ["Technology"] * 3 + ["Nature"] * 3

# Compute embeddings
emb = model.encode(sentences_diverse)

# Reduce to 2D
pca = PCA(n_components=2)
emb_2d = pca.fit_transform(emb)

print(f"Original shape: {emb.shape}")
print(f"After PCA: {emb_2d.shape}")
print(f"Variance explained: {sum(pca.explained_variance_ratio_)*100:.1f}%")

In [ ]:
# Visualize
fig, ax = plt.subplots(figsize=(10, 8))

colors = {'Animals': 'red', 'Technology': 'blue', 'Nature': 'green'}

for cat in set(categories):
    mask = [c == cat for c in categories]
    ax.scatter(emb_2d[mask, 0], emb_2d[mask, 1], 
               c=colors[cat], label=cat, s=150, alpha=0.7)

# Add labels
for i, sent in enumerate(sentences_diverse):
    short = sent[:20] + "..." if len(sent) > 20 else sent
    ax.annotate(short, (emb_2d[i, 0], emb_2d[i, 1]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

ax.set_xlabel('PCA Component 1', fontsize=11)
ax.set_ylabel('PCA Component 2', fontsize=11)
ax.set_title('Embedding Space (2D projection)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 🤔 Question 4

Look at the 2D visualization:

1. Do sentences from the same category cluster together?
2. Which category seems most tightly clustered? Why might that be?
3. PCA only explains a fraction of the variance. What does this tell us about the embedding space?

---

# Part 6: Semantic Search — A Practical Application

One of the most powerful applications of embeddings is **semantic search**:

Given a query, find the most similar items in a database.

Unlike keyword search, semantic search understands **meaning**.

In [ ]:
# Our "database" of scientific topics
database = [
    "CRISPR gene editing technology for treating genetic diseases",
    "Protein folding prediction using deep learning models",
    "Climate change impact on marine ecosystems",
    "Quantum computing algorithms for optimization problems",
    "Drug discovery using molecular simulations",
    "Machine learning for analyzing medical images",
    "Renewable energy storage in batteries",
    "Neuroimaging techniques for brain research",
]

# Pre-compute database embeddings
db_embeddings = model.encode(database)

print(f"Database: {len(database)} items")
print(f"Embedding shape: {db_embeddings.shape}")

In [ ]:
def semantic_search(query, top_k=3):
    """Find the most similar items to a query."""
    # Embed the query
    query_emb = model.encode([query])
    
    # Compute similarities with all database items
    similarities = cosine_similarity(query_emb, db_embeddings)[0]
    
    # Get top-k indices
    top_indices = similarities.argsort()[::-1][:top_k]
    
    print(f"Query: '{query}'\n")
    print("Most similar items:")
    for rank, idx in enumerate(top_indices, 1):
        print(f"  {rank}. [{similarities[idx]:.3f}] {database[idx]}")

# Try a search
semantic_search("How can AI help with biology?")

### Exercise 1: Try Your Own Searches

Modify the query below and run the search. Try queries related to your research field!

In [ ]:
# TODO: Try different queries!
my_query = "neural networks for medical diagnosis"  # <-- Modify this!

semantic_search(my_query)

### 🤔 Question 5

1. Try a query that doesn't use any exact words from the database (e.g., "AI for healthcare"). Does it still find relevant results?
2. Why does semantic search work even without keyword matching?
3. Can you think of cases where semantic search might fail or give unexpected results?

---

# Summary: Key Takeaways

## What We Learned

1. **Embeddings are dense vector representations** where similar things have similar vectors

2. **The pipeline**: Raw data → Tokens → Token IDs → Embeddings

3. **Cosine similarity** is the standard way to measure embedding similarity

4. **Pre-trained models** (like Sentence-BERT) have learned powerful embeddings from massive datasets

5. **Applications**: Semantic search, clustering, classification, recommendation

## Key Insight

> **Meaning becomes geometry.** When you have good embeddings, reasoning about meaning becomes reasoning about distances and directions in vector space.

## What's Next

In **Part 2**, we'll explore embeddings for different scientific domains:
- Text embeddings for scientific abstracts
- Molecular embeddings (SMILES)
- Protein embeddings (amino acid sequences)
- DNA embeddings (nucleotide sequences)

---

## Reflection Questions

Before moving to Part 2, think about:

1. **In your research domain**, what data do you work with that could be embedded? (text, sequences, structures, images?)

2. **What would "similarity" mean** in your domain? What should be close in embedding space?

3. **What tasks** could you solve if you had good embeddings for your data? (search, clustering, classification?)